In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import cv2 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from keras.wrappers.scikit_learn import KerasClassifier

import pathlib

##############################
uniq_labels=['yes','no']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# New Section

In [ ]:
#!unzip -uq "/content/archive (3).zip" -d "/content/drive/My Drive/archive"
directory = "/content/drive/My Drive/archive"

In [ ]:
num_classes = len(classes_names_list)
print(num_classes)

In [ ]:
DATA_PATH = os.path.join(directory, "brain_tumor_dataset")
data_dir_list = os.listdir(DATA_PATH)

import cv2

classes_names_list=[]
img_data_list=[]

for dataset in data_dir_list:
    classes_names_list.append(dataset) 
    print ('Loading images from {} folder\n'.format(dataset)) 
    img_list=os.listdir(DATA_PATH+'/'+ dataset)
    for img in img_list:
        input_img=cv2.imread(DATA_PATH + '/'+ dataset + '/'+ img )
        input_img_resize=cv2.resize(input_img,(240, 240))
        (b, g, r)=cv2.split(input_img_resize) 
        img=cv2.merge([r,g,b])
        img_data_list.append(img)

Loading images from no folder

Loading images from yes folder



In [ ]:
import numpy as np

img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data /= 255

print (img_data.shape)
num_of_samples = img_data.shape[0]
input_shape = img_data[0].shape

classes = np.ones((num_of_samples,), dtype='int64')

classes[0:98]=0
classes[98:]=1

from keras.utils import to_categorical

classes = to_categorical(classes, 2)

from sklearn.utils import shuffle

X, Y = shuffle(img_data, classes, random_state=456)


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=456)

In [ ]:
def create_model():
    model = Sequential()

####### Model
    model.add(Conv2D(64, (7, 7), strides = (1,1) , padding="same", activation ='relu', input_shape=X_train.shape[1:]))
    model.add(MaxPooling2D(pool_size=(4, 4)))

    model.add(Conv2D(32, (7, 7), strides = (1,1),padding="same", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Dense(2))
    model.add(Activation('softmax'))
    print(model.summary())     
    
    #initialize the training data augmentation object
    trainAug = keras.preprocessing.image.ImageDataGenerator(rotation_range=15, fill_mode="nearest")

    # Compiling the model
    opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
    model.compile(loss="categorical_crossentropy", optimizer= opt, metrics=["accuracy"])#categorical_crossentropy,binary_crossentropy
    # Training and evaluating the model
    #validation_acc = np.amax(history['val_acc']) 
    #print('Best validation acc of epoch:', validation_acc)
    return model

In [ ]:
model = create_model()

stop_early = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20, verbose=1 )    
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2 , callbacks = stop_early)


In [ ]:
import numpy as np

img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data /= 255

print (img_data.shape)
num_of_samples = img_data.shape[0]
input_shape = img_data[0].shape

classes = np.ones((num_of_samples,), dtype='int64')

from keras.utils import to_categorical

#classes = to_categorical(classes, 2)

from sklearn.utils import shuffle

#X_s, Y_s = shuffle(img_data, classes, random_state=456)


X_trainS, X_testS, y_trainS, y_testS = train_test_split(img_data,classes, test_size=0.2, random_state=456)



In [ ]:
def create_model_GS():
    model = Sequential()

####### Model
    model.add(Conv2D(64, (7, 7), strides = (1,1) , padding="same", activation ='relu', input_shape=X_trainS.shape[1:]))
    model.add(MaxPooling2D(pool_size=(4, 4)))

    model.add(Conv2D(32, (7, 7), strides = (1,1),padding="same", activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Dense(2))
    model.add(Activation('softmax'))
    print(model.summary())     
    
    #initialize the training data augmentation object
    trainAug = keras.preprocessing.image.ImageDataGenerator(rotation_range=15, fill_mode="nearest")

    # Compiling the model
    opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
    model.compile(loss="categorical_crossentropy", optimizer= opt, metrics=["accuracy"])#categorical_crossentropy,binary_crossentropy
    # Training and evaluating the model
    #validation_acc = np.amax(history['val_acc']) 
    #print('Best validation acc of epoch:', validation_acc)
    return model

In [ ]:
from sklearn.model_selection import GridSearchCV
from keras.layers import Dropout
from sklearn.model_selection import KFold
KFold

KC = KerasClassifier(build_fn = create_model_GS,verbose = 0,batch_size = 40,epochs = 10)

parameters = {'batch_size' : [25,32],
          'epochs' : [5,10]}
grid_search = GridSearchCV(estimator=KC , 
param_grid=parameters,scoring='accuracy',cv=10)


grid_search.fit(X_trainS,y_trainS)
# Summarize the results

print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{},{} with: {}'.format(mean, stdev, param))


In [ ]:
model.compile(loss="categorical_crossentropy", optimizer= opt, metrics=["accuracy"])#categorical_crossentropy,binary_crossentropy
# Training and evaluating the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2 , callbacks = stop_early)

In [ ]:
def plot_random_examples():

  plt.figure( dpi=256)
  predicted_classes = model.predict_classes(X_test)

  for i in range(10):
    plt.subplot(5,5,i+1)
    fig=plt.imshow(X_test[i,:,:,:])
    plt.axis('off')
    plt.title("Predicted - {}".format(uniq_labels[predicted_classes[i]]),fontsize=3)
    plt.tight_layout()
    
plot_random_examples()

In [ ]:
pd.DataFrame(history.history).plot(figsize=(12, 8))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()

In [ ]:
model_evaluate = model.evaluate(X_test, y_test)
print("Loss     : ",model_evaluate[0])
print("accuracy : ",model_evaluate[1])

In [ ]:
y_pred = model.predict_classes(X_test)
#real values for test images
y_test_=np.argmax(y_test, axis=1)


In [ ]:
print("Classification report : \n",classification_report(y_test_, y_pred))

In [ ]:
cm = confusion_matrix(y_test_, y_pred)
cm

In [ ]:
def draw_confusion_matrix(true,preds):
    # Compute confusion matrix
    conf_matx = confusion_matrix(true, preds)
    print("Confusion matrix : \n")
    sns.heatmap(conf_matx, annot=True,annot_kws={"size": 12},fmt='g', cbar=False, cmap="viridis")
    plt.show()
    return conf_matx
con_mat = draw_confusion_matrix(y_test_, y_pred)

In [ ]:
model = Sequential()

model.add(Conv2D(64, (4, 2), padding='same', strides = (1,1), activation='relu', input_shape=X_train.shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(32, (2, 2), padding='same', strides = (1,1), activation='relu'))
model.add(Dropout(0.25))

model.add(Conv2D(64, (4, 2), padding='same',strides = (1,1), activation='relu', input_shape=X_train.shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(32, (2, 2), padding='same', strides = (1,1), activation='relu'))
model.add(Dropout(0.25))

model.add(Conv2D(64, (4, 2), padding='same', strides = (1,1), activation='relu', input_shape=X_train.shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(32, (2, 2), padding='same',strides = (1,1), activation='relu'))
model.add(Dropout(0.25))

model.add(Conv2D(64, (4, 2), padding='same',strides = (1,1), activation='relu', input_shape=X_train.shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(32, (2, 2), padding='same',strides = (1,1), activation='relu'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Dense(2))
model.add(Activation('softmax'))
print(model.summary())

In [ ]:
opt = keras.optimizers.Adam(lr=0.00001, beta_1=0.8, beta_2=0.9)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

stop_early = keras.callbacks.EarlyStopping(monitor='val_acc', patience=4, verbose=1 )

history=model.fit(X_train, y_train,
          batch_size=32,
          epochs=10,
          validation_split=0.15,
          callbacks=[stop_early],
          shuffle=True) 

In [ ]:
def plot_random_examples():

  plt.figure( dpi=256)
  predicted_classes = model.predict_classes(X_test)

  for i in range(10):
    plt.subplot(5,5,i+1)
    fig=plt.imshow(X_test[i,:,:,:])
    plt.axis('off')
    plt.title("Predicted - {}".format(uniq_labels[predicted_classes[i]]),fontsize=3)
    plt.tight_layout()
    
plot_random_examples()

In [ ]:
pd.DataFrame(history.history).plot(figsize=(12, 8))
plt.grid(True)
plt.gca().set_ylim(0, 1) # set the vertical range to [0-1]
plt.show()

In [ ]:
y_pred = model.predict_classes(X_test)
#real values for test images
y_test_=np.argmax(y_test, axis=1)